In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('default')
import numpy as np
import xarray as xr
from ftplib import FTP
import urllib
import pandas as pd

# Real Iceberg

In [2]:
year = 2015 # 2002 through 2015 available
iip_url_base = 'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G00807/' 
iip_filename = 'IIP_{}IcebergSeason.csv'.format(year)
iip_url = iip_url_base + iip_filename

In [3]:
r = urllib.request.urlretrieve(iip_url)
df = pd.read_csv(r[0])

In [4]:
n = df.BERG_NUMBER.mode()[0]

In [5]:
real_berg = df.loc[df['BERG_NUMBER'] == n]

In [6]:
real_berg_track = real_berg.loc[6318:6333]

# Input Fields

## Ocean

### GLBv0.08

- 3 hr
- 0.08 degree

In [7]:
glb_url = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_56.3'

In [8]:
glb_ds = xr.open_dataset(glb_url, decode_times=False)

In [9]:
# June 1, 00:00 = 135120., July 1, 00:00 = 135840.
glb_ds = glb_ds.sel(depth=0.0, lat = slice(45., 51.), lon = slice(-53., -47.), time=slice(135120., 135840.))

In [10]:
water_u = glb_ds.water_u
water_v = glb_ds.water_v
water_temp = glb_ds.water_temp

In [11]:
glb_times = np.asarray(water_u.time)
glb_lats = np.asarray(water_u.lat)
glb_lons = np.asarray(water_u.lon)

In [12]:
glb_times = glb_times - 135120

###

In [13]:
water_u = np.asarray(water_u)
water_v = np.asarray(water_v)
water_temp = np.asarray(water_temp)

In [ ]:
glb_times

## Atmosphere

### NAVGEM

- 6 hr
- 0.08 degree

In [ ]:
navgem_url = 'http://coastwatch.pfeg.noaa.gov/erddap/griddap/erdNavgem05D10mWind_LonPM180'

In [ ]:
navgem_ds = xr.open_dataset(navgem_url)

In [ ]:
navgem_ds = navgem_ds.sel(time=slice('2015-06-01','2015-07-01'), latitude=slice(45,51), longitude=slice(-53,-47))

In [ ]:
wind_u = navgem_ds.wnd_ucmp_height_above_ground
wind_v = navgem_ds.wnd_vcmp_height_above_ground

In [ ]:
#navgem_times = np.asarray(wind_u.time)
navgem_times = np.linspace(0, 120*6 - 6, 120)
navgem_lats = np.asarray(wind_u.latitude)
navgem_lons = np.asarray(wind_u.longitude)

In [ ]:
wind_u = np.asarray(wind_u)
wind_v = np.asarray(wind_v)

# Tools

In [ ]:
def find_nearest(val, arr):
    diff_arr = abs(arr - val)
    smallest_ind = np.argpartition(diff_arr, 0)[0]
    smallest_val = arr[smallest_ind]
    next_smallest_ind = np.argpartition(diff_arr, 1)[1]
    next_smallest_val = arr[next_smallest_ind]
    return smallest_ind, smallest_val, next_smallest_ind, next_smallest_val       

In [ ]:
def linear_interp_1D(x, x1, x2):  
    xi1 = (x2 - x)/(x2 - x1)
    xi2 = (x - x1)/(x2 - x1)
    xi = x1*xi1 + x2*xi2
    return xi

# Model

In [ ]:
import scipy.io as sio
import numpy.matlib
import cmath

In [ ]:
# Constants
R = 6378*1e3
om = 7.2921e-5
rhow = 1027
rhoa = 1.2
rhoi = 850
drho = rhow - rhoi
Cw = 0.9
Ca = 1.3
gam = np.sqrt(rhoa*drho/rhow/rhoi*(Ca/Cw))
sst0 = -4
Cs1 = 1.5; Cs2 = 0.5; Cs3 = 0.1
CMv1 = 7.62e-3; CMv2 = 1.29e-3; CMe1 = 0.5
CMb1 = 0.58; CMb2 = 0.8; CMb3 = 0.2

In [ ]:
# Timesteps
t0 = 360  # initial hour
t_inc = 3  # 3 hr
num_days = 15
tn = t_inc * num_days  # total number of timesteps
dt = 1*3*3600  # model timestep in seconds
tf = t0 + tn*dt  # final timestep in seconds  
t_all = np.arange(t0, 720, 3)

In [ ]:
t_all

In [ ]:
# Iceberg Inits
x0, y0 = -50.14, 48.49   # lon, lat
l0, w0, h0 = 600, 500, 400   # length, width, height

In [ ]:
def main():

    x, y = x0, y0
    l, w, h = l0, w0, h0
    t = t0
    iceberg = np.array([[t0],[x0],[y0],[l0],[w0],[h0]])

    while t < max(t_all):

        x_new, y_new, l_new, w_new, h_new = iceDEF(t, x, y, l, w, h)

        
        if x_new > -47 or x_new < -53 or y_new > 51 or y_new < 45:
            # Iceberg out-of-bounds
            print(x_new)
            print(y_new)
            print('out-of-bounds')
            break
        

        if l_new <= 0 or w_new <= 0 or h_new <= 0:
            # Iceberg melted
            print('melted')
            break

        else:
            x, y, l, w, h = x_new, y_new, l_new, w_new, h_new
            t += t_inc
            iceberg_new = np.array([[t],[x],[y],[l],[w],[h]])
            iceberg = np.column_stack((iceberg, iceberg_new))

    plt.plot(real_berg_track['LONGITUDE'], real_berg_track['LATITUDE'], label='observed')        
    plt.plot(iceberg[1,:],iceberg[2,:], label='computed')
    plt.legend()

        
    save_dict = {
                    't_arr': iceberg[0,:],
                    'x_arr': iceberg[1,:],
                    'y_arr': iceberg[2,:],
                    'l_arr': iceberg[3,:],
                    'w_arr': iceberg[4,:],
                    'h_arr': iceberg[5,:]
                }

    

def iceDEF(t,x,y,l,w,h):

    print('\ntimestep: {}\n'.format(t))
    

    # Atmosphere

    t_atm_ind1, t_atm_val1, t_atm_ind2, t_atm_val2 = find_nearest(t, navgem_times)
    x_atm_ind1, x_atm_val1, x_atm_ind2, x_atm_val2 = find_nearest(t, navgem_lons)
    y_atm_ind1, y_atm_val1, y_atm_ind2, y_atm_val2 = find_nearest(t, navgem_lats)
    
    #(t, height, lat, lon)
    vau = wind_u[t_atm_ind1, 0, y_atm_ind1, x_atm_ind1] 
    vav = wind_v[t_atm_ind1, 0, y_atm_ind1, x_atm_ind1] 

    
    # Ocean
    
    t_ocean_ind1, t_ocean_val1, t_ocean_ind2, t_ocean_val2 = find_nearest(t, glb_times)
    x_ocean_ind1, x_ocean_val1, x_ocean_ind2, x_ocean_val2 = find_nearest(t, glb_lons)
    y_ocean_ind1, y_ocean_val1, y_ocean_ind2, y_ocean_val2 = find_nearest(t, glb_lats)
    
    #(t, lat, lon)
    vwu = water_u[t_ocean_ind1, y_ocean_ind1, x_ocean_ind1] 
    vwv = water_v[t_ocean_ind1, y_ocean_ind1, x_ocean_ind1] 
    sst = water_temp[t_ocean_ind1, y_ocean_ind1, x_ocean_ind1] 

    print('vau = ', vau)
    print('vav = ', vav)
    print('vwu = ', vwu)
    print('vwv = ', vwv)
    print('sst = ', sst)
    

    # Drifting

    S = np.pi*((l*w)/(l+w))
    ff = 2*om*np.sin((np.abs(y)*np.pi)/180)
    lam = np.sqrt(2)*Cw*(gam*np.sqrt(vau**2 + vav**2))/(ff*S)
    print('S = {0:.15f}'.format(S))
    print('ff = {0:.15f}'.format(ff))
    print('lam = {0:.15f}'.format(lam))
    
    if lam < 0.1:
        print('Taylor approx used for alpha')
        alpha = lam*(lam**4*(lam**4*(lam**4*(-0.0386699020961393*lam**4 + \
            0.055242717280199) - 0.0883883476483184) + \
            0.176776695296637) - 0.707106781186548)
    else:
        alpha = np.multiply(np.divide(np.sqrt(2),np.power(lam, 3)),(1-np.sqrt(1+np.power(lam,4))))
        
    if lam < 0.6:
        print('Taylor approx used for beta')
        beta = lam**3*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*(lam**4*\
            (0.0153268598203613*lam**4 - 0.0151656272365985) + \
            0.0180267866272764) + 0.0219176256311202) - \
            0.0274446790511418) + 0.0357675015202851) - \
            0.0493731785691779) + 0.0745776683282687) - \
            0.132582521472478) + 0.353553390593274)
    else:
        beta = np.real(np.multiply(np.divide(1.,np.power(lam,3.)),cmath.sqrt(np.multiply((4.+np.power(lam,4.)), \
            cmath.sqrt(1.+np.power(lam,4.)))-3.*np.power(lam,4.)-4.)))

    print('alpha = {0:.15f}'.format(alpha))
    print(beta)
    print('beta = {}'.format(beta))

    viu = vwu + gam*(-alpha*vav + beta*vau)
    viv = vwv + gam*(alpha*vau + beta*vav)

    print('viu = {0:.15f}'.format(viu))
    print('viv = {0:.15f}'.format(viv))

    y_new = y + (viv*dt)*(180/(np.pi*R))
    x_new = x + (viu*dt)/(np.cos((((y + y_new)/2)*np.pi)/180))*(180/(np.pi*R))

    print('x_new = {0:.15f}'.format(x_new))
    print('y_new = {0:.15f}'.format(y_new))
    
    # Melting

    Me = CMe1*(Cs1*np.sqrt(vau**2 + vav**2)**Cs2 + Cs3*np.sqrt(vau**2 + vav**2))
    Mv = CMv1*sst + CMv2*sst**2
    Mb = CMb1*np.power(np.sqrt(np.square(viu-vwu)+np.square(viv-vwv)),CMb2)*(sst - sst0)/l**CMb3

    print('Me = {0:.15f}'.format(Me))
    print('Mv = {0:.15f}'.format(Mv))
    print('Mb = {0:.15f}'.format(Mb))

    l_new = l - (Mv + Me)*(dt/(24*3600))  # convert dt from secs to days
    w_new = w - (Mv + Me)*(dt/(24*3600))
    h_new = h - Mb*(dt/(24*3600))

    if w_new < 0.85*h_new:
        # Rollover
        print('rollover')
        w_new, h_new = h_new, w_new

    if w_new > l_new:
        # Ensure l is greater than w
        print('swap l and w')
        w_new, l_new = l_new, w_new

    print('l_new = {0:.15f}'.format(l_new))
    print('w_new = {0:.15f}'.format(w_new))
    print('h_new = {0:.15f}'.format(h_new))

    return x_new, y_new, l_new, w_new, h_new    


if __name__=="__main__":
    main()